In [3]:
import pandas as pd

In [136]:
gamesAll = pd.read_csv("games.csv", index_col=0)

In [137]:
gamesAll.head()

,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
GAME_DATE_EST,,,,,,,,,,,,,,,,,,,,
2022-12-22,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,0.382,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
2022-12-22,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,0.457,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2022-12-21,22200466,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,0.313,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1
2022-12-21,22200467,Final,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,0.297,27.0,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1
2022-12-21,22200468,Final,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,0.378,22.0,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0


In [20]:
gamesAll.shape

(26651, 20)

In [22]:
games2022=gamesAll[gamesAll["SEASON"]==2022]

In [26]:
games2022.shape

(542, 20)

In [33]:
games2022.dtypes

GAME_ID               int64
GAME_STATUS_TEXT     object
HOME_TEAM_ID          int64
VISITOR_TEAM_ID       int64
SEASON                int64
TEAM_ID_home          int64
PTS_home            float64
FG_PCT_home         float64
FT_PCT_home         float64
FG3_PCT_home        float64
AST_home            float64
REB_home            float64
TEAM_ID_away          int64
PTS_away            float64
FG_PCT_away         float64
FT_PCT_away         float64
FG3_PCT_away        float64
AST_away            float64
REB_away            float64
HOME_TEAM_WINS        int64
dtype: object

In [65]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [186]:
#sets number of dedcisions trees to 50
rf=RandomForestClassifier(n_estimators=50,min_samples_split=10,random_state=1)

In [41]:
train=gamesAll[gamesAll["SEASON"]<2022]

In [42]:
test=gamesAll[gamesAll["SEASON"]==2022]

In [196]:
# clean the data
float_cols = gamesAll.select_dtypes(include=['float32']).columns

# Replace infinite values with NaN
gamesAll[float_cols] = gamesAll[float_cols].replace([np.inf, -np.inf], np.nan)

# Drop rows with any NaN values
gamesAll = gamesAll.dropna()

# Define a threshold for excessively large values (this threshold can be adjusted as needed)
threshold = np.finfo(np.float32).max

# Remove rows with values too large for dtype float32
gamesAll = gamesAll[(gamesAll[float_cols] < threshold).all(axis=1)]

# Identify and drop object columns
object_cols = gamesAll.select_dtypes(include=['object']).columns
gamesAll = gamesAll.drop(columns=object_cols)

# Verify there are no NaN or infinite values left in the entire DataFrame
assert gamesAll.isnull().sum().sum() == 0, "There are still NaN values in the data"
assert np.isfinite(gamesAll).all().all(), "There are still infinite values in the data"

# Define predictors and target
predictors = ["VISITOR_TEAM_ID", "FG3_PCT_home","REB_home","PTS_home"]
target = "HOME_TEAM_WINS"

# Ensure the columns exist in the DataFrame
assert all(col in gamesAll.columns for col in predictors), "One or more predictor columns are missing"
assert target in gamesAll.columns, "Target column is missing"

# Verify there are no NaN, infinite, or excessively large values in the predictor columns
for col in predictors:
    assert gamesAll[col].isnull().sum() == 0, f"NaN values found in column {col}"
    assert np.isfinite(gamesAll[col]).all(), f"Infinite values found in column {col}"
    assert (gamesAll[col] < threshold).all(), f"Excessively large values found in column {col}"



In [197]:
# train on seasons prior to 2021, test on 2022
train = gamesAll[gamesAll["SEASON"]<2021]
test = gamesAll[gamesAll["SEASON"]>2021]
rf.fit(train[predictors], train[target])


RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [198]:
preds=rf.predict(test[predictors])

In [199]:
from sklearn.metrics import accuracy_score

In [200]:
acc=accuracy_score(test["HOME_TEAM_WINS"],preds)

In [201]:
acc

0.6494464944649446

In [109]:
combined = pd.DataFrame(dict(actual=test["HOME_TEAM_WINS"], predicted=preds))


In [110]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])


predicted,0,1
actual,,
0,54,169
1,46,273


In [111]:
from sklearn.metrics import precision_score


In [112]:
precision_score(test["HOME_TEAM_WINS"], preds)


0.6176470588235294

In [121]:
group_matches=gamesAll.groupby("HOME_TEAM_ID")

KeyError: '1610612740'

In [155]:
def computeAverage(group,cols,new_cols):
        rolling_stats = group[cols].rolling(3, closed='left').mean()
        group[new_cols] = rolling_stats
        group = group.dropna(subset=new_cols)
        return group

In [156]:
cols=["PTS_home","PTS_away","FG_PCT_home","FG_PCT_away","AST_home","AST_away"]
new_cols = [f"{c}_rolling" for c in cols]



In [157]:
matches_rolling = gamesAll.groupby("HOME_TEAM_ID").apply(lambda x: computeAverage(x, cols, new_cols))


In [158]:
matches_rolling

GAME_ID  HOME_TEAM_ID  VISITOR_TEAM_ID  SEASON  \
HOME_TEAM_ID GAME_DATE_EST                                                    
1610612737   2022-12-05     22200354    1610612737       1610612760    2022   
             2022-12-02     22200327    1610612737       1610612743    2022   
             2022-11-27     22200293    1610612737       1610612748    2022   
             2022-11-23     22200263    1610612737       1610612758    2022   
             2022-11-19     22200235    1610612737       1610612761    2022   
...                              ...           ...              ...     ...   
1610612766   2014-10-29     21400004    1610612766       1610612749    2014   
             2014-10-23     11400107    1610612766       1610612754    2014   
             2014-10-15     11400053    1610612766       1610612765    2014   
             2014-10-13     11400043    1610612766       1610612753    2014   
             2014-10-10     11400027    1610612766       1610612764    2014   

                            TEAM_ID_home  PTS_home  FG_PCT_home  FT_PCT_home  \
HOME_TEAM_ID GAME_DATE_EST                                                     
1610612737   2022-12-05       1610612737     114.0        0.431        0.765   
             2022-12-02       1610612737     117.0        0.557        0.643   
             2022-11-27       1610612737      98.0        0.398        0.947   
             2022-11-23       1610612737     115.0        0.446        0.957   
             2022-11-19       1610612737     124.0        0.500        0.781   
...                                  ...       ...          ...          ...   
1610612766   2014-10-29       1610612766     108.0        0.406        0.690   
             2014-10-23       1610612766      79.0        0.370        0.808   
             2014-10-15       1610612766      84.0        0.341        0.862   
             2014-10-13       1610612766      99.0        0.426        0.800   
             2014-10-10       1610612766      99.0        0.427        0.696   

                            FG3_PCT_home  AST_home  ...  FG3_PCT_away  \
HOME_TEAM_ID GAME_DATE_EST                          ...                 
1610612737   2022-12-05            0.342      24.0  ...         0.353   
             2022-12-02            0.357      24.0  ...         0.375   
             2022-11-27            0.278      24.0  ...         0.273   
             2022-11-23            0.367      23.0  ...         0.290   
             2022-11-19            0.280      23.0  ...         0.216   
...                                  ...       ...  ...           ...   
1610612766   2014-10-29            0.286      27.0  ...         0.500   
             2014-10-23            0.222      18.0  ...         0.417   
             2014-10-15            0.136      15.0  ...         0.400   
             2014-10-13            0.150      24.0  ...         0.357   
             2014-10-10            0.464      26.0  ...         0.333   

                            AST_away  REB_away  HOME_TEAM_WINS  \
HOME_TEAM_ID GAME_DATE_EST                                       
1610612737   2022-12-05         25.0      49.0               0   
             2022-12-02         31.0      33.0               1   
             2022-11-27         28.0      46.0               0   
             2022-11-23         27.0      48.0               1   
             2022-11-19         27.0      51.0               1   
...                              ...       ...             ...   
1610612766   2014-10-29         25.0      35.0               1   
             2014-10-23         21.0      46.0               0   
             2014-10-15         28.0      58.0               0   
             2014-10-13         16.0      45.0               1   
             2014-10-10         22.0      44.0               1   

                            PTS_home_rolling  PTS_away_rolling  \
HOME_TEAM_ID GAME_DATE_EST                                       
1610612737   2022-1

In [159]:
gamesAll.head()

,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
GAME_DATE_EST,,,,,,,,,,,,,,,,,,,
2022-12-22,22200477,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,0.382,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
2022-12-22,22200478,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,0.457,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2022-12-21,22200466,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,0.313,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1
2022-12-21,22200467,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,0.297,27.0,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1
2022-12-21,22200468,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,0.378,22.0,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0


In [164]:
def make_predictions(data, predictors):
    train = gamesAll[gamesAll["SEASON"]<2021]
    test = gamesAll[gamesAll["SEASON"]>2021]
    rf.fit(train[predictors], train[target])
    preds=rf.predict(test[predictors])    
    combined = pd.DataFrame(dict(actual=test["HOME_TEAM_WINS"], predicted=preds), index=test.index)
    error = precision_score(test["HOME_TEAM_WINS"], preds)
    return combined, error

In [163]:
matches_rolling.head()

GAME_ID  HOME_TEAM_ID  VISITOR_TEAM_ID  SEASON  \
HOME_TEAM_ID GAME_DATE_EST                                                    
1610612737   2022-12-05     22200354    1610612737       1610612760    2022   
             2022-12-02     22200327    1610612737       1610612743    2022   
             2022-11-27     22200293    1610612737       1610612748    2022   
             2022-11-23     22200263    1610612737       1610612758    2022   
             2022-11-19     22200235    1610612737       1610612761    2022   

                            TEAM_ID_home  PTS_home  FG_PCT_home  FT_PCT_home  \
HOME_TEAM_ID GAME_DATE_EST                                                     
1610612737   2022-12-05       1610612737     114.0        0.431        0.765   
             2022-12-02       1610612737     117.0        0.557        0.643   
             2022-11-27       1610612737      98.0        0.398        0.947   
             2022-11-23       1610612737     115.0        0.446        0.957   
             2022-11-19       1610612737     124.0        0.500        0.781   

                            FG3_PCT_home  AST_home  ...  FG3_PCT_away  \
HOME_TEAM_ID GAME_DATE_EST                          ...                 
1610612737   2022-12-05            0.342      24.0  ...         0.353   
             2022-12-02            0.357      24.0  ...         0.375   
             2022-11-27            0.278      24.0  ...         0.273   
             2022-11-23            0.367      23.0  ...         0.290   
             2022-11-19            0.280      23.0  ...         0.216   

                            AST_away  REB_away  HOME_TEAM_WINS  \
HOME_TEAM_ID GAME_DATE_EST                                       
1610612737   2022-12-05         25.0      49.0               0   
             2022-12-02         31.0      33.0               1   
             2022-11-27         28.0      46.0               0   
             2022-11-23         27.0      48.0               1   
             2022-11-19         27.0      51.0               1   

                            PTS_home_rolling  PTS_away_rolling  \
HOME_TEAM_ID GAME_DATE_EST                                       
1610612737   2022-12-05           119.000000        119.000000   
             2022-12-02           121.000000        122.666667   
             2022-11-27           118.000000        117.333333   
             2022-11-23           109.666667        112.000000   
             2022-11-19           110.000000        107.000000   

                            FG_PCT_home_rolling  FG_PCT_away_rolling  \
HOME_TEAM_ID GAME_DATE_EST                                             
1610612737   2022-12-05                0.473333             0.484667   
             2022-12-02                0.474000             0.469000   
             2022-11-27                0.489333             0.487333   
             2022-11-23                0.462000             0.490000   
             2022-11-19                0.467000             0.492667   

                            AST_home_rolling  AST_away_rolling  
HOME_TEAM_ID GAME_DATE_EST                                      
1610612737   2022-12-05            26.333333         25.666667  
             2022-12-02            27.000000         27.333333  
             2022-11-27            25.333333         28.000000  
             2022-11-23            24.000000         28.000000  
             2022-11-19            23.666667         28.666667  

[5 rows x 25 columns]

In [181]:
# Step 1: Drop 'level_0' column if it exists
combined = combined.drop(columns=['level_0'], errors='ignore')

# Step 2: Reset index and ensure no conflict with existing columns
combined = combined.reset_index(drop=True)

# Step 3: Drop 'HOME_TEAM_ID' and 'VISITOR_TEAM_ID' if they exist
combined = combined.drop(columns=['HOME_TEAM_ID', 'VISITOR_TEAM_ID'], errors='ignore')

# Step 4: Reset index for matches_rolling
matches_rolling = matches_rolling.reset_index(drop=True)

# Step 5: Rename columns in matches_rolling to avoid conflicts
matches_rolling = matches_rolling.rename(columns={
    "VISITOR_TEAM_ID": "VISITOR_TEAM_ID_new",
    "HOME_TEAM_ID": "HOME_TEAM_ID_new",
    "HOME_TEAM_WINS": "HOME_TEAM_WINS_new"
})

# Step 6: Merge DataFrames
combined = combined.merge(
    matches_rolling[["VISITOR_TEAM_ID_new", "HOME_TEAM_ID_new", "HOME_TEAM_WINS_new"]],
    left_index=True, right_index=True
)

In [182]:
combined.head(10)


,index,GAME_DATE_EST,actual,predicted,VISITOR_TEAM_ID_new,HOME_TEAM_ID_new,HOME_TEAM_WINS_new
0,0,2022-12-22,1,1,1610612760,1610612737,0
1,1,2022-12-22,1,1,1610612743,1610612737,1
2,2,2022-12-21,1,0,1610612748,1610612737,0
3,3,2022-12-21,1,1,1610612758,1610612737,1
4,4,2022-12-21,0,1,1610612761,1610612737,1
5,5,2022-12-21,0,1,1610612738,1610612737,0
6,6,2022-12-21,1,0,1610612755,1610612737,1
7,7,2022-12-21,0,0,1610612762,1610612737,0
8,8,2022-12-21,0,1,1610612749,1610612737,1
9,9,2022-12-21,0,1,1610612740,1610612737,1


In [185]:
# check accuracy of predictions for each team
grouped = combined.groupby('HOME_TEAM_ID_new')
grouped.head()

,index,GAME_DATE_EST,actual,predicted,VISITOR_TEAM_ID_new,HOME_TEAM_ID_new,HOME_TEAM_WINS_new
0,0,2022-12-22,1,1,1610612760,1610612737,0
1,1,2022-12-22,1,1,1610612743,1610612737,1
2,2,2022-12-21,1,0,1610612748,1610612737,0
3,3,2022-12-21,1,1,1610612758,1610612737,1
4,4,2022-12-21,0,1,1610612761,1610612737,1
